#### DBSCAN and Use-case for exploring anomalic instances:

In [1]:
# tabular manipulation:
import numpy as np
import pandas as pd
# visualization:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt
import seaborn as sns
# sklearn for scaling and clustering:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
# environment:
from env import host, user, password

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

url = get_db_url("grocery_db")

sql = """
select *
from grocery_customers
"""

df = pd.read_sql(sql, url, index_col="customer_id")
df.head()


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 30.9 KB


#### We will need to proceed through several operational steps to achieve utility from DBSCAN:

- Select what variables/features that we wish to examine
- Scale these features (DBSCAN is going to be useful for continuous variables)
- Ensure that our features are in a numpy array for fitting DBSCAN
- Select our epsilon and min_neighbors to fit our clusters
- Use our clusters to label outliers
- Explore our clusters

db scan is expecting numpy array, take .values out will change it to numpy array.

In [3]:
df.columns

Index(['Channel', 'Region', 'Fresh', 'Milk', 'Grocery', 'Frozen',
       'Detergents_Paper', 'Delicassen'],
      dtype='object')

## Select Our Features:

In [4]:
selected_features = ['Fresh','Milk', 'Grocery']
#note choosing continous variables is going to be significantly more valuable for distance based clustering as
#poiints in space will not inherently snap to any given set values and density will mean more as a result.

## Scale Our Features

In [ ]:
#we will treat df as oof it has been split into a train, validate, test and we are examining train.

In [5]:
# We will create min max scaler:
# make it 
# fit it 
# use it



#creating the scaler
minmax = MinMaxScaler()



In [7]:
#scaling the features
scaled_features = minmax.fit_transform(df[selected_features])

In [8]:
#examine our numpy array.
scaled_features

array([[0.11294004, 0.13072723, 0.08146416],
       [0.06289903, 0.13282409, 0.10309667],
       [0.05662161, 0.11918086, 0.08278992],
       ...,
       [0.1295431 , 0.21013575, 0.32594285],
       [0.091727  , 0.02622442, 0.02402535],
       [0.02482434, 0.02237109, 0.02702178]])

In [9]:
#check if its array, it is
#this is suitable for feeding into out DBSCAN model.
type(scaled_features)

numpy.ndarray

In [10]:
# glue the scaled info back into the df using concat:
scaled_cols = [col + '_scaled' for col in selected_features]

In [11]:
#creating df which is the scaled data
scaled_df = pd.DataFrame(scaled_features, index = df.index, columns = scaled_cols)

In [13]:
#combining the scaled df with original df
df = df.merge(scaled_df, on = df.index)

In [15]:
df.head()

,key_0,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Fresh_scaled,Milk_scaled,Grocery_scaled
0,0,2,3,12669,9656,7561,214,2674,1338,0.112940,0.130727,0.081464
1,1,2,3,7057,9810,9568,1762,3293,1776,0.062899,0.132824,0.103097
2,2,2,3,6353,8808,7684,2405,3516,7844,0.056622,0.119181,0.082790
3,3,1,3,13265,1196,4221,6404,507,1788,0.118254,0.015536,0.045464
4,4,2,3,22615,5410,7198,3915,1777,5185,0.201626,0.072914,0.077552


## Creating Our DBSCAN

In [ ]:
#NOW WE WILL UTILIZE

In [16]:
# Creating our DBSCAN model:
#make it, fit it, use it

In [17]:
df.shape

(440, 12)

#### *Make It!*

In [22]:
#with minmax scaler only see vaules between 0 and 1.
# we choose eps of 0.1 beacuase you havd to make a guess then go from there the smaller (more strict) the more condensed the cluster will be the higher the value the less condensed it becomes
# min_samples was choosen based on size, starting off small then adjusting.
dbsc = DBSCAN(eps = 0.1,min_samples = 20)

#### *Fit It!*

In [23]:
#will feed in the scaled features, fit on scaled!
dbsc.fit(scaled_features)

DBSCAN(eps=0.1, min_samples=20)

#### *Use It!*

In [24]:
# we will look at quality of our actual model
# -1 is not defined within the cluster
dbsc.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0, -1,  0, -1,  0, -1,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0, -1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -1, -1,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0, -1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [25]:
#CREATING A FEATURE WHICH UTILLIZED THE LABELS_
df['labels'] = dbsc.labels_

In [27]:
#looking at our select features 
df[selected_features + ['labels']].head()

,Fresh,Milk,Grocery,labels
0,12669,9656,7561,0
1,7057,9810,9568,0
2,6353,8808,7684,0
3,13265,1196,4221,0
4,22615,5410,7198,0


In [29]:
# 31 outliers, only have 1 cluster here.
# example 0,1,2 would be three clusters.
#main takeaway cluster or not a cluster!!!
df.labels.value_counts()

 0    409
-1     31
Name: labels, dtype: int64

In [ ]:
# We can now Explore

# *Explore*

In [32]:
sns.scatterplot(x = 'Fresh' , y= 'Milk' , hue = 'labels', data = df)

<AxesSubplot:xlabel='Fresh', ylabel='Milk'>

In [33]:
sns.scatterplot(x = 'Fresh' , y= 'Grocery' , hue = 'labels', data = df)

<AxesSubplot:xlabel='Fresh', ylabel='Grocery'>

In [ ]:
# lets examine it on a 3d Scale

In [37]:
fig = plt.figure(1, figsize = (10,10))
ax = Axes3D(fig)
ax.scatter(df.Fresh,
          df.Milk,
          df.Grocery,
          c = df.labels,
          edgecolor = 'k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

ax.set_xlabel('Fresh')
ax.set_ylabel('Milk')
ax.set_zlabel('Grocery')

Text(0.5, 0, 'Grocery')

In [ ]:
# Initial Takeaways:
# - We have at least ~10 points thats are definite outliers
# - We have some data points that may or may not belong to the main cluster
# - Out of the dimensions we observed there does not appear to be need for more than one cluster.
# - We may want to be less strict on hyperparamters to cathc the data points that arent as extreme as the 
#         furthest outliers in the dataset 